In [1]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Reusable classes
import sys
sys.path.append("/home/jovyan/work/upload/ml-experiments/sc_wrangling")

# Data path
import pathlib
FOLDER = pathlib.Path("/home/jovyan/work/upload/")
TABLE_PATH = FOLDER / 'data/pickle/df_columns_labels.pkl'
LAW2VEC_PATH = FOLDER / 'data/law2vec/Law2Vec.200d.txt'

# Math stuff
import numpy as np
from numpy.core.records import ndarray

# Manage ML lifecycle
import mlflow
from mlflow import log_params, set_tags, log_metrics
MLFLOW_TRACKING_URI = 'http://srv.meaningfy.ws:8989'

# Data visualisation
import pandas as pd
from pandas import Series

# Vectorize algotrithm
from gensim.models import Word2Vec, KeyedVectors

# Support Vector Classifier Algorithm
from sklearn.svm import SVC

# Intermediate steps of the pipeline must be ‘transforms’,
# that is, they must implement fit and transform methods.
from sklearn.pipeline import Pipeline

# Splitting data into train and test
from sklearn.model_selection import train_test_split

# Metrics Evaluation Methods
from evaluation_metrics import model_evaluation_metrics

In [2]:
# Read data
df = pd.read_pickle(TABLE_PATH)
# Independent data
columns = df['Concatenated Data (clean)']
# Label data
category = df['Type of measure']

## law2vec model

In [3]:
# Load a word2vec model stored in the C *text* format.
model = KeyedVectors.load_word2vec_format(LAW2VEC_PATH, binary=False)
l2v_dict = {w: vec for w, vec in zip(model.wv.index2word, model.wv.syn0)}

In [4]:
class MeanEmbeddingVectorizer(object):
    """Calculate the mean of each word"""
    def __init__(self, word2vec):
        self.word2vec = word2vec
        if len(word2vec)>0:
            self.dim=len(word2vec[next(iter(l2v_dict))])
        else:
            self.dim=0

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0) for words in X]
        )

In [68]:
# Use pipes to implement steps of fit and transform method
svm_l2v = Pipeline([
    # Add the words we want to mean
    ("law2vec vectorizer", MeanEmbeddingVectorizer(l2v_dict)),
    # Use SVC algorithm
    ("SVM", SVC(gamma=1, C=5))])

## Train SVM Model

In [38]:
# split data into test and train sets
# where "train" contains 70% of data and "test" - 30%
X_train, X_test, y_train, y_test = train_test_split(
    columns, category, random_state=42, test_size=0.3, shuffle=True)

In [69]:
# Fit SVM model with out train data
svm_l2v.fit(X_train, y_train)

Pipeline(steps=[('law2vec vectorizer',
                 <__main__.MeanEmbeddingVectorizer object at 0x7fb048be44f0>),
                ('SVM', SVC(C=5, gamma=1))])

In [70]:
# Model Evaluation
svm_l2v.score(X_train, y_train)

0.963302752293578

In [71]:
# SVM prediction based on test data
prediction = svm_l2v.predict(X_test)
prediction

array([0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 3, 0, 1, 1, 0, 2, 0,
       0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 2, 0, 0, 2, 0, 2, 2, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0,
       0, 2, 0, 0, 5, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       2, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 3, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 2, 0, 2, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 2, 2, 0, 3, 0, 0, 2, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0,
       1, 3, 0, 0, 1, 0, 4, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

## Evaluation

In [72]:
model_evaluation_metrics(y_test, prediction, 'Category')

,Evaluation Metrics,Category
0,Accuracy,0.818505
1,Precission,0.682853
2,Recall,0.456966
3,F1 Score,0.500542
4,Mean Absolute Error,0.373665
5,Mean Squared Error,1.021352


In [73]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(experiment_name="Type of measure (Law2Vec)")

with mlflow.start_run():

    parameters = {"Language model": 'law2vec',
                  "Random state": 42,
                  "Test size": 0.3,
                  "Gamma": 1,
                  "C": 5}
    log_params(parameters)
    metrics = { "Accuracy": 0.818505,
            "Precision": 0.682853,
            "Recall": 0.456966,
            "F1 Score": 0.500542,
            "Mean Absolute Error": 0.373665,
            "Mean Squared Error": 1.021352}
    log_metrics(metrics)